In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np

#This header will be added to requests to identify the user to the web admin
headers = {'user-agent': 'Alex Sylvester/Vancouver, alexander.d.sylvester@gmail.com'}

urls = [
    'https://vancouver.craigslist.org/search/van/ct', #vancouver 
    'https://vancouver.craigslist.org/search/pml/ct', #tricities
    'https://vancouver.craigslist.org/search/rch/ct', #richmond
    'https://vancouver.craigslist.org/search/nvn/ct', #north shore
    'https://vancouver.craigslist.org/search/rds/ct', #delta surrey langley
    'https://vancouver.craigslist.org/search/bnc/ct', #burnaby/newwest
    'https://comoxvalley.craigslist.org/search/ct', #comox valley
    'https://abbotsford.craigslist.org/search/ct', #fraser valley
    'https://kamloops.craigslist.org/search/ct', #kamloops
    'https://kelowna.craigslist.org/search/ct', #kelowna
    'https://nanaimo.craigslist.org/search/ct', #nanaimo
    'https://sunshine.craigslist.org/search/ct', #sunshine coast
    'https://victoria.craigslist.org/search/ct', #victoria
]

own_deal = {'o?':'owner', 'd?':'dealer'} #add this at the end of the url to specify owner or dealer sale

In [2]:
import pandas as pd
from time import sleep #to avoid bombarding the webpage with requests
from random import randint #avoid throttling by not sending too many requests one after the other

car_df = pd.DataFrame({}) #create an empty dataframe to hold the all the car data

for url in urls: #loop though base urls
    for od in own_deal: #loop though owner or dealer sales
        #get the first craigslist page for the url (with price above 100)
        response = requests.get(url + od + 'min_price=100', headers=headers)
        soup = BeautifulSoup(response.text, 'lxml')#covert to beutiful soup object

        #the following few lines are from this tutorial: https://towardsdatascience.com/web-scraping-craigslist-a-complete-tutorial-c41cea4f4981
        #find the total number of posts to find the limit of the pagination
        results_num = soup.find('div', class_= 'search-legend')
        results_total = int(results_num.find('span', class_='totalcount').text) #pulled the total count of posts as the upper bound of the pages array

        #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
        pages = np.arange(0, results_total+1, 120)

        for page in pages:
            print('page:', page)

            #get request
            response = requests.get(url
                           + od
                           + "s=" #the parameter for defining the page number 
                           + str(page) #the page number in the pages array from earlier
                           + "&min_price=100") #only get entries with price above $100

            #sleep(1) #wait one second between reqests
            soup = BeautifulSoup(response.text, 'lxml')#covert to beutiful soup object
            entries = soup.find_all('li', class_='result-row') #get a list of entries on the page

            for entry in entries: #loop over the cars on the page
                try: #in case an error is thrown on some vehicle, this will stop the whole process from failing
                    price = entry.find('span', class_='result-price').text #price of the car
                    title = entry.find('a', class_='result-title hdrlnk') #get the title/link for the entry
                    car_url = title['href'] #get the url from the title

                    #sleep(1) #wait one second between reqests
                    car = requests.get(car_url) #get the web page for the car 
                    car_soup = BeautifulSoup(car.text, 'lxml') #convert the webpage to a soup object

                    location = car_soup.find(attrs={'name': 'geo.placename'})['content'] #location (eg. city)
                    latlong = car_soup.find(attrs={'name': 'geo.position'})['content'] #latitude and longitude in the post
                    body = car_soup.find('section', id = 'postingbody').text #body test of posting

                    all_attrs = car_soup.find_all('p', class_='attrgroup') #all of the vehicle attribute groups
                    make = all_attrs[0].b.text #first attribute group is the title
                    print(make)
                    attrs = all_attrs[1].find_all('span') #the rest of the attributes are grouped together
                    attrs = [attr.text for attr in attrs] #convert attributes to a list

                    car_lib = {attr.split(':')[0]: attr.split(':')[1] for attr in attrs if ':' in attr} #library to hold the attributes
                    car_lib['make'] = make #add the title to the library
                    car_lib['price'] = price #add the price to the library
                    car_lib['sale type'] = own_deal[od] #type of sale (owner or dealer)
                    car_lib['location'] = location
                    car_lib['latlong'] = latlong
                    car_lib['body text'] = body 

                    car_df = car_df.append(car_lib, ignore_index=True) #append the car data to the dataframe
                except:
                    pass

car_df.drop('VIN', axis=1, inplace=True) #drop the VIN column to keep the data anonynmous
car_df

page: 0
2007 bmw x3
1984 honda accord
2008 gmc c7500
2003 acura rsx
2005 acura rsx
2002 bmw 530i
2001 Acura TL 3.2L
2005 nissan altima
1997 toyota 4runner
2006 chrysler 300
2001 Honda Civic
1990 Murcury Topez
1985 mercury grand marquis
2011 toyota venza xle awd
2006 lexus rx400h
2014 mazda cx-5
2002 BMW x5
1999 toyota corolla
2008 smart car passion
2001 toyota corolla
1981 cadillac eldorado
2013 Mini Cooper Countryman
2001 honda civic si
2008 hyundai accent
2012 Toyota RAV4
2017 Ram 2500 Cummins
2004 Toyota Alphard
2002 BMW X5 4.4L
2021 Tesla Model Y
1941 Willys
2004 Toyota Corolla
2015 toyota corolla
2000 bmw
2004 gmc safari van
1997 Jeep TJ
2005 Toyota Celica GT
2011 mercedes-benz ml350 4matic
2005 lincoln town car
2005 pontiac vibe
2009 bmw 328i convertible
2010 hyundai santa fe
2005 Toyota Celica GT
2016 ford focus
2020 Toyota Tacoma
2001 mercedes benz c240
2000 bmw
2002 International
2006 Ford Mustang V6 Convertible
1993 gmc safari
2000 bmw
2008 bmw 328i
2000 bmw
2000 bmw
2003 Hin

2005 Toyota Crown Majesta
2004 infinity g35
1983 porsche 944
2000 GMC 3500
2000 GMC 3500
2013 mini cooper countryman s all4
2007 2007 pontiac g5
2008 audi tt coupe 2.0t
2002 2002 mazda protege 5 Hatchback
1998 toyota camry
2001 honda civic
2017 bmw m3 sedan
2014 acura mdx
2012 hyundai sonata hybrid
2015 toyota camry hybrid xle
2015 hyundai veloster
2009 Subaru WRX
1966 ford econoline cargo van
2018 Alfa Romeo 4C
2010 volkswagen golf
2014 Dodge Ram 1500 4x4
2008 chevrolet aveo
2010 Suzuki Grand Vitara
2008 Jaguar
2007 VW Jetta 2.5
2007 toyota tacoma 4x4
2015 volkswagen jetta
2003 pontiac montana van
2015 nissan micra sv
1999 ford explorer
1977 porsche 911s
2013 mazda2
2019 lincoln mkz hybrid
2013 chevrolet silverado 2500
2017 ford transit connect xlt
2010 hyundai elantra touring
2017 mercedes benz
2009 nissan versa sl
2014 bmw 428i coupe
2015 ford fiesta se
2009 toyota yaris sedan
2007 chevrolet aveo
2012 toyota rav4
2012 bmw x5
2016 Ford focus
2019 mercedes-benz e450
2013 fiat 500 spor

2002 Acura MDX
2008 mercedes benz.  C300
2003 bmw 540i
2000 volvo v40
2005 2005 Blue Bird
2000 Hino 3200
2011 kia sorento
2017 mercedes benz B250 4MATIC
2009 Chevrolet corvette z06
1999 Nissan Pathfinder LE
2007 mazda mazdaspeed3
2014 scion xd
2006 porsche cayenne
1988 Lamborghini countach
1998 1998 gmc sierra
2003 Volkswagen Passat
2008 BMW 335i convertible
1971 1971 Dodge Dart
2006 acura tl 3.2 4dr sedan
2011 nissan versa
1987 Chevrolet regular cab
2006 volkswagen jetta tdi
2015 lexus is 350 f sport awd
1996 toyota landcruiser hzj73
2010 chevrolet cobalt ls
2007 Pontiac g5
2000 Saab 9-3 Viggen
2008 Land Rover LR2 SE
2019 Tesla
2019 Freightliner 108SD
2004 jeep grand cherokee
2010 mini cooper s
2019 dodge ram 1500 laramie
2000 BMW Z 3
2013 audi a7 3.0t premium plus
2020 bmw 750li
2010 acura mdx
2005 mercedes benz
2000 Ford
page: 840
2000 Ford
2009 volkswagen cc
1975 Gmc pickup
2010 mazda 3
2013 nissan sentra sr
1964 Chevrolet chevelle
2016 jeep wrangler unlimited
2013 mercedes benz
20

2006 Smart Fortwo
2003 Toyota Funcargo
2011 Volkswagen Jetta
2020 kia sportage
page: 1200
2020 Lemon
1997 1997 Acura
2007 Food Truck
2012 Honda CRZ
2010 hyundai genesis coupe 2.0t
2003 volkswagen jetta wagon
2005 Honda Elysion
2009 infiniti ex35 awd
2015 audi q5 technik
2015 bmw x1 xdrive35i
2008 saturn astra
2018 bmw m2
2011 Ford F150 XLT 4x4
2000 Acura 3.2 tl
2006 Ford Ranger
2011 cadillac srx
2010 mercedes benz
2000 Chevrolet  Venture
2017 bmw x5
1971 1971 DODGGE CHALLENGER R/T
1992 toyota paseo
2006 audi s4 avant
2012 hyundai accent
2008 jeep wrangler jk unlimited
2008 Mercedes Benz e300
2010 2010 acura csx
2015 hyundai veloster
2010 smart ForTwo passion
2007 dodge grand caravan
1987 Rolls-Royce Silver Spur
1996 Nissan Atlas
2019 ford f-150
2009 cadillac escalade esv
1980 maserati quattroporte
2012 kia optima sx turbo
2015 bmw 328i
2005 jeep liberty
2008 volvo xc90 3.2
2012 2012 ford f-150
2017 mercedes c300
2005 2005 chevy colorado
1999 ford explorer
2019 Mercedes Benz E53 AMG
200

2009 Ford focus se
2007 mazda 5
2004 volvo xc70
1967 Chrysler 300 convertible
2010 Volkswagen Tiguan
2000 honda acty
2005 honda civic coupe
2009 E90 BMW 323i
2003 bmw 525i
2005 ford mustang
2010 nissan altima
2008 2008 Mercedes Benz S65 AMG V12
2012 honda civic
2015 hundai elantra
2011 Ford f150
1997 buick lesabre limited
1992 1992 Civic DX
2019 nissan micra sv
2010 bmw 328i
2000 ford taurus
1986 Ford F150
1999 ford mustang
2013 Subaru BRZ Sport-Tech
1969 barracuda notchback coupe
2018 audi s3
2014 jeep grand cherokee laredo 4wd
2016 toyota tacoma limited
2016 volkswagen jetta
2017 ford transit passenger 350 xlt
1990 ford e150
2016 Bmw
1997 jeep grand cherokee
2006 toyota corolla
1999 bmw m3
1997 Porsche 993
2011 bmw 323i
1975 TVR
2002 DODGE CARAVAN SE
2003 lexus is300
2008 infiniti g35
2006 chrysler 300c hemi
2012 Honda Civic LX
2014 toyota camry
2005 jaguar x-type 3.0 awd
2002 pontiac sunfire
2003 vw jetta GLI
2003 Honda Civic
2010 audi a5 2.0t quattro
2010 kia soul
2015 Dodge Dart G

...

2003 Ford Super Duty F-350 SRW
2015 Dodge Grand Caravan
2005 Ford Mustang
2007 Ford F-150
2012 Ford Focus
2013 Ford F-150
2007 Chevrolet Corvette
2013 Dodge Grand Caravan
2010 Chevrolet Cobalt
2014 Chevrolet Cruze
2015 Chevrolet Sonic
2015 Chevrolet Cruze
2004 BMW 3-Series
2007 Acura RDX
2008 Chevrolet Silverado 2500HD
2004 Volvo XC90 T6 AWD
2008 Chevrolet Silverado 1500
2009 Chevrolet Silverado 1500
2019 Tesla Model 3
2009 Chevrolet Express Cargo Van
2010 Chevrolet Malibu
2011 Chevrolet Silverado 1500
2002 Mazda B2300
2012 Chevrolet Equinox
2015 dodge ram cummins 3500 laramie
2019 Tesla Model 3
2014 ford f150 xlt 4wd
2013 Toyota Tacoma SR5
2015 Nissan Rogue S
2016 Nissan Sentra-$7995
2013 Dodge Grand Caravan
2006 subaru legacy
2018 Ram Ram Pickup 1500
2018 subaru wrx sti
2016 volkswagen golf r
2015 honda accord hybrid touring
2018 volkswagen tiguan comfortline
2019 bmw x1 xdrive28i
2016 jeep grand cherokee srt
2018 acura mdx sh-awd tech pkg
2019 ford transit connect
2011 lexus is 250 

2018 bmw x6 xdrive35i
2018 nissan titan sv
2019 Ram 1500
2020 mazda cx-3
2008 smart fortwo
2014 Kia Rio
2018 DODGE JOURNEY
2021 chevrolet blazer lt
2017 chevrolet tahoe lt
2019 gmc acadia slt awd
2018 gmc sierra 1500
2021 gmc savana 3500 box truck
2018 acura mdx sh-awd tech pkg
2019 Chevrolet Trax
2011 Ford Fiesta SE
2019 Kia Niro EX
2020 Kia Sorento LX+ AWD
2018 Kia Soul
2019 Kia Niro
2018 Mercedes-Benz GLS 63 AMG
2018 Kia Soul EX+
2018 Kia Forte EX
2017 honda accord
2014 Kia Rio LX+
2014 Kia Rondo Lx
2014 Honda Civic
2017 Hyundai Accent GL
2017 subaru wrx
2017 Kia Soul EV
2018 Mitsubishi Outlander PHEV GT
2013 ford focus titanium
2021 Sierra 3500HD Duramax Denali
2010 Toyota Avalon XLS
2019 2019 Chevrolet Impala
2018 BMW X6 XDrive35I
2019 2019 Chevrolet Malibu
2018 2018 Chevrolet Sonic
2019 2019 Honda Civic
2018 Honda Civic
2017 nissan rogue sl awd
2017 nissan rogue sl platinum
2017 nissan murano platinum
2017 nissan gt-r premium
2015 honda accord hybrid touring
2014 honda civic si c

2019 Subaru Outback 3.6L Premier
2019 Subaru Outback 2.5 Premier
2020 Outback Touring
2018 Subaru Forester 2.0XT Touring
2018 chevrolet traverse high country
2017 buick encore sport touring
2020 Subaru Forester Premier
2020 2020 Subaru Forester
2019 kia sorento ex awd
2005 Mazda Mazda3
2020 Mazda CX-30
2009 Volkswagen GTI
2008 Toyota Tacoma
2009 Toyota Matrix
2010 Toyota Tacoma
2006 Mercedes-Benz CLS-Class
2012 Mazda Mazdaspeed 3
2011 Jeep Grand Cherokee
2013 nissan altima sv
2020 nissan 370z
2018 nissan rogue sl platinum
2016 nissan rogue sl
2013 mitsubishi rvr
2009 BMW 335I Xdrive AWD
2009 nissan sentra
2014 dodge journey
2018 Jeep Renegade Trailhawk
2017 toyota corolla
2019 kia sorento
2020 mitsubishi rvr
2012 ford mustang
2018 toyota c-hr
2018 Acura RDX Tech
2014 chrysler 200
2019 gmc sierra 1500
2018 chevrolet spark
2014 dodge grand caravan sxt
2016 cadillac ct6 3.6 v6 awd
2008 chevrolet express 3500 cargo van
2017 vw volkswagen Golf R
2005 mercedes benz c230
2009 mercedes benz c2

2018 Mazda CX-5 GS AWD
2016 Mazda CX-5 GS AWD
2010 Mazda Tribute GT
2017 Mazda 6 GS
2019 Mazda CX-3 GS
2016 Mazda 3 GT Sedan
2018 Tundra SR5 Plus
2018 dodge ram 1500 4x4
2018 ford f-150 limited
2016 HYUNDAI TUCSON LIMITED
2013 nissan juke sv
2019 audi a5
2011 BMW 323i
2014 Dodge Dart SE
2014 Hyundai Accent GL
2020 model
2020 model
2020 model
2020 model
2018 model
2014 HONDA CIVIC
2020 model
2018 FORD FUSION TITANIUM
2019 Hyundai Tucson Preferred
2019 jaguar e-pace s
2016 Chevrolet Spark LS Hatchback
2018 Equinox LS AWD 2.4L
2018 2018 JEEP
2018 Tesla Model 3
2016 Toyota Venza
2016 Ram 1500
2008 Porsche 911
2016 Dodge Journey
2016 Ram 1500
2019 Subaru BRZ
2016 Nissan Rogue
2014 Nissan Altima
2016 Dodge Journey
2019 GMC Savana
2016 Dodge Journey
2019 Mitsubishi RVR
2019 Mitsubishi Eclipse Cross
2019 Mitsubishi Eclipse Cross
2019 Mitsubishi Eclipse Cross
2019 Mitsubishi RVR
2019 Mitsubishi Outlander PHEV
2019 Mitsubishi Outlander PHEV
2017 Chevrolet Bolt EV
2014 Ram Cargo Van
2019 Mitsubis

,body text,condition,drive,fuel,latlong,location,make,odometer,paint color,price,sale type,title status,transmission,type,cylinders,size
0,\n\nQR Code Link to This Post\n\n\n2007 BMW X3...,good,4wd,gas,49.234834;-123.090563,VANCOUVER,2007 bmw x3,281000,white,"$5,500",owner,rebuilt,automatic,SUV,NaN,NaN
1,\n\nQR Code Link to This Post\n\n\nRuns and dr...,NaN,NaN,gas,49.283710;-123.124466,Vancouver,1984 honda accord,140000,NaN,$600,owner,clean,automatic,NaN,NaN,NaN
2,\n\nQR Code Link to This Post\n\n\n2008\nGmc C...,excellent,rwd,diesel,49.210453;-123.108330,Vancouver,2008 gmc c7500,110000,white,"$67,000",owner,clean,automatic,truck,8 cylinders,NaN
3,"\n\nQR Code Link to This Post\n\n\nHello, I am...",NaN,NaN,gas,49.232300;-123.087400,Vancouver,2003 acura rsx,243000,NaN,"$4,000",owner,clean,automatic,NaN,NaN,NaN
4,"\n\nQR Code Link to This Post\n\n\nHello, I ha...",salvage,NaN,gas,49.220000;-123.071000,Vancouver,2005 acura rsx,225000,black,"$1,500",owner,salvage,automatic,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37336,\n\nQR Code Link to This Post\n\n\n2020 *** Ma...,excellent,fwd,gas,0.000000;0.000000,Port Coquitlam,2020 Mazda Mazda6,28356,black,"$28,995",dealer,clean,automatic,NaN,4 cylinders,NaN
37337,\n\nQR Code Link to This Post\n\n\n2020 *** To...,excellent,fwd,gas,0.000000;0.000000,Port Coquitlam,2020 Toyota Camry,23820,black,"$27,995",dealer,clean,automatic,NaN,4 cylinders,NaN
37338,\n\nQR Code Link to This Post\n\n\n2016 *** Fo...,excellent,4wd,gas,0.000000;0.000000,Port Coquitlam,2016 Ford F-150,129449,white,"$29,800",dealer,clean,automatic,NaN,8 cylinders,NaN
37339,\n\nQR Code Link to This Post\n\n\n2020 *** Fo...,excellent,4wd,gas,0.000000;0.000000,Port Coquitlam,2020 Ford Escape,31512,silver,"$28,995",dealer,clean,automatic,NaN,3 cylinders,NaN


In [3]:
#store car data in a csv file
car_df.to_csv('cars.csv')